valuation methods
fundamental metrics, PE, DE
analyst price price predictions


| # | Model                              | Input Frequency     | Output                |
| - | ---------------------------------- | ------------------- | --------------------- |
| 1 | **Multi-Stage DCF**                | Quarterly (TTM FCF) | Fair price per share  |
| 2 | **Adjusted Present Value (APV)**   | Quarterly           | Fair price per share  |
| 3 | **FCFE Model**                     | Quarterly           | Fair price per share  |
| 4 | **Dividend Discount (Gordon)**     | Quarterly           | Fair price per share  |
| 5 | **Residual Income (RI)**           | Quarterly           | Fair price per share  |
| 6 | **EVA-Based Valuation**            | Quarterly           | Fair price per share  |
| 7 | **Inverted Fama–French 5 Factor**  | Daily factors       | Required return $k_i$ |
| 8 | **Inverted q-Factor (Hou et al.)** | Daily factors       | Required return $k_i$ |


2. End-to-End Pipeline
Data Ingestion

Prices & factors: Daily price, FF5 & q-factor premia, T-bill rate

Fundamentals: Quarterly filings → compute TTM FCF, EPS, Book Value, NOPAT, Capital, Dividends, Share Count

Model Calculations (Quarterly Update)

DCF/APV/FCFE/Dividend/RI/EVA: Use latest TTM inputs + consensus or 5-yr historical growth (“g”) + dynamically estimated WACC (from yield curves + credit spreads)

Output: 6 fair-price estimates per stock

Idee om eerst alles apart te testen om soort feature importance te bepalen

# Task
Combine the income statement, balance sheet, and cash flow annual reports for IBM into a single dataframe and display it.

## Extract annual reports

### Subtask:
Extract the list of annual reports from the `income_statement`, `balance_sheet`, and `cash_flow` dictionaries.


**Reasoning**:
Extract the annual reports lists from the fetched financial data dictionaries.



# **ratio's**

**📊 Liquidity Ratios
Measure a company’s ability to meet short-term obligations. **

Current Ratio	Current Assets / Current Liabilities	Measures short-term solvency.

Quick Ratio (Acid-Test)	(Current Assets - Inventory) / Current Liabilities	Measures liquidity without relying on inventory.

Cash Ratio	(Cash + Cash Equivalents) / Current Liabilities	Most conservative liquidity ratio.


**💰 Profitability Ratios
Evaluate a company’s ability to generate profit. **



Gross Profit Margin	Gross Profit / Revenue × 100	Shows profitability after cost of goods sold.

Operating Profit Margin	Operating Profit / Revenue × 100	Measures core business profitability.

Net Profit Margin	Net Income / Revenue × 100	Final profitability after all expenses.

Return on Assets (ROA)	Net Income / Total Assets × 100	Efficiency in using assets to generate profit.

Return on Equity (ROE)	Net Income / Shareholders’ Equity × 100	Measures return on shareholder investment.

Return on Capital Employed (ROCE)	EBIT / Capital Employed × 100	Efficiency of capital use.

⚖️ Solvency (Leverage) Ratios
Assess long-term financial stability.

Debt-to-Equity Ratio	Total Debt / Total Equity	Measures financial leverage.

Equity Ratio	Equity / Total Assets × 100	Indicates the proportion of assets financed by equity.

Interest Coverage Ratio	EBIT / Interest Expense	Shows how easily interest payments are covered.

Debt Ratio	Total Liabilities / Total Assets	Proportion of assets financed by debt.

📈 Efficiency (Activity) Ratios
Measure how well assets and liabilities are managed.

Inventory Turnover	Cost of Goods Sold / Average Inventory	Measures inventory efficiency.

Accounts Receivable Turnover	Net Credit Sales / Average Accounts Receivable	Shows how quickly receivables are collected.

Accounts Payable Turnover	Cost of Goods Sold / Average Accounts Payable	Efficiency in paying suppliers.

Asset Turnover Ratio	Revenue / Average Total
Assets	Shows how effectively assets generate sales.

Working Capital Turnover	Revenue / Average Working Capital	Efficiency in using working capital.


**💵 Market Ratios (Valuation Ratios)
Used to assess stock market performance and investor returns.**


Earnings Per Share (EPS)	(Net Income - Preferred Dividends) / Weighted Avg. Shares	Shows profitability per share.

Price-to-Earnings (P/E)	Market Price per Share / EPS	Indicates stock valuation.

Price-to-Book (P/B)	Market Price per Share / Book Value per Share	Compares market and book value.

Dividend Yield	Annual Dividends per Share / Price per Share × 100	Measures dividend income vs stock price.

Dividend Payout Ratio	Dividends / Net Income × 100	Proportion of earnings paid to shareholders.

Cash flow models

over or undervaluation

# Data Download

This section downloads financials for the top 5 NASDAQ companies by market capitalization (approx.): `AAPL`, `MSFT`, `NVDA`, `AMZN`, `GOOGL`.
It uses the existing Alpha Vantage fetch helper and collects: Company Overview, Income Statement, Balance Sheet, and Cash Flow.



In [125]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# Symbols to analyze (top 5 NASDAQ by market cap; static list for simplicity)
symbols = ['AAPL','MSFT','NVDA','AMZN','GOOGL']

# Reuse existing helper
fetch = fetch_alpha_vantage_data
API_KEY = API_KEY

company_data = {}
for sym in symbols:
    try:
        ov = fetch('OVERVIEW', sym, API_KEY)
        is_ = fetch('INCOME_STATEMENT', sym, API_KEY)
        bs = fetch('BALANCE_SHEET', sym, API_KEY)
        cf = fetch('CASH_FLOW', sym, API_KEY)
        company_data[sym] = {'overview': ov, 'income': is_, 'bs': bs, 'cf': cf}
    except Exception as e:
        company_data[sym] = {'error': str(e)}

# Brief check
display(Markdown(f"Downloaded data for: {', '.join(company_data.keys())}"))


Downloaded data for: AAPL, MSFT, NVDA, AMZN, GOOGL

# Ratio Calculations

This section computes Liquidity, Profitability, Solvency, Efficiency, and Market ratios for each symbol using annual data (latest year) to create a company-by-ratio table.
Missing values are handled gracefully and annotated as N/A.



In [126]:
# Build one-row per company with ratios (using latest annual reports)

def to_float(x):
    try:
        return float(x)
    except Exception:
        return np.nan

rows = []
for sym in symbols:
    data = company_data.get(sym, {})
    ov = data.get('overview', {}) if isinstance(data.get('overview', {}), dict) else {}
    is_ann = (data.get('income', {}) or {}).get('annualReports', [])
    bs_ann = (data.get('bs', {}) or {}).get('annualReports', [])
    cf_ann = (data.get('cf', {}) or {}).get('annualReports', [])

    is0 = is_ann[0] if len(is_ann) > 0 else {}
    bs0 = bs_ann[0] if len(bs_ann) > 0 else {}

    # Core numbers
    revenue = to_float(is0.get('totalRevenue'))
    gross_profit = to_float(is0.get('grossProfit'))
    operating_income = to_float(is0.get('operatingIncome'))
    ebit = to_float(is0.get('ebit')) if is0.get('ebit') not in (None, 'None', '') else to_float(is0.get('operatingIncome'))
    net_income = to_float(is0.get('netIncome'))
    interest_expense = to_float(is0.get('interestExpense')) if is0.get('interestExpense') not in (None, 'None', '') else to_float(is0.get('interestAndDebtExpense'))

    cur_assets = to_float(bs0.get('totalCurrentAssets'))
    cur_liab = to_float(bs0.get('totalCurrentLiabilities'))
    inventory = to_float(bs0.get('inventory'))
    cash_eq = to_float(bs0.get('cashAndCashEquivalentsAtCarryingValue'))
    if np.isnan(cash_eq):
        cash_eq = to_float(bs0.get('cashAndShortTermInvestments'))
    total_assets = to_float(bs0.get('totalAssets'))
    total_liab = to_float(bs0.get('totalLiabilities'))
    equity = to_float(bs0.get('totalShareholderEquity'))

    total_debt = to_float(bs0.get('shortLongTermDebtTotal'))
    if np.isnan(total_debt):
        total_debt = to_float(bs0.get('longTermDebt')) + to_float(bs0.get('shortTermDebt'))

    # Market
    price_per_share = np.nan
    if ov.get('MarketCapitalization') and ov.get('SharesOutstanding'):
        price_per_share = to_float(ov['MarketCapitalization']) / max(to_float(ov['SharesOutstanding']), 1.0)
    eps_ttm = to_float(ov.get('DilutedEPSTTM') or ov.get('EPS'))
    book_value_per_share = to_float(ov.get('BookValue'))

    # Liquidity
    current_ratio = cur_assets / cur_liab if cur_liab and cur_liab != 0 else np.nan
    quick_ratio = (cur_assets - inventory) / cur_liab if cur_liab and cur_liab != 0 else np.nan
    cash_ratio = cash_eq / cur_liab if cur_liab and cur_liab != 0 else np.nan

    # Profitability
    gross_margin = (gross_profit / revenue * 100) if revenue else np.nan
    op_margin = (operating_income / revenue * 100) if revenue else np.nan
    net_margin = (net_income / revenue * 100) if revenue else np.nan
    roa = (net_income / total_assets * 100) if total_assets else np.nan
    roe = (net_income / equity * 100) if equity else np.nan
    roce = (ebit / (total_assets - cur_liab) * 100) if (total_assets and cur_liab and (total_assets - cur_liab) != 0) else np.nan

    # Solvency
    debt_to_equity = (total_debt / equity) if equity else np.nan
    equity_ratio = (equity / total_assets * 100) if total_assets else np.nan
    interest_cov = (ebit / interest_expense) if interest_expense not in (None, 0, np.nan) else np.nan
    debt_ratio = (total_liab / total_assets) if total_assets else np.nan

    # Efficiency (annual; averages not available cross-year here, so mark N/A)
    inv_turn = np.nan
    ar_turn = np.nan
    ap_turn = np.nan
    asset_turn = (revenue / total_assets) if total_assets else np.nan
    wc_turn = np.nan

    # Market ratios
    pe_ratio = (price_per_share / eps_ttm) if eps_ttm not in (None, 0, np.nan) else np.nan
    pb_ratio = (price_per_share / book_value_per_share) if book_value_per_share not in (None, 0, np.nan) else np.nan

    rows.append({
        'symbol': sym,
        # Liquidity
        'current_ratio': current_ratio,
        'quick_ratio': quick_ratio,
        'cash_ratio': cash_ratio,
        # Profitability
        'gross_profit_margin_pct': gross_margin,
        'operating_profit_margin_pct': op_margin,
        'net_profit_margin_pct': net_margin,
        'roa_pct': roa,
        'roe_pct': roe,
        'roce_pct': roce,
        # Solvency
        'debt_to_equity': debt_to_equity,
        'equity_ratio_pct': equity_ratio,
        'interest_coverage': interest_cov,
        'debt_ratio': debt_ratio,
        # Efficiency
        'inventory_turnover': inv_turn,
        'ar_turnover': ar_turn,
        'ap_turnover': ap_turn,
        'asset_turnover': asset_turn,
        'working_capital_turnover': wc_turn,
        # Market
        'pe_ratio': pe_ratio,
        'pb_ratio': pb_ratio
    })

ratios_table = pd.DataFrame(rows).set_index('symbol')

# Display, formatting
with pd.option_context('display.float_format', lambda v: f"{v:,.2f}"):
    display(Markdown('### Company Ratios (latest annual)'))
    display(ratios_table)

# Note for missing efficiency ratios
display(Markdown("_Note: Some efficiency ratios left N/A due to lack of average quarterly values across years._"))


### Company Ratios (latest annual)

,current_ratio,quick_ratio,cash_ratio,gross_profit_margin_pct,operating_profit_margin_pct,net_profit_margin_pct,roa_pct,roe_pct,roce_pct,debt_to_equity,equity_ratio_pct,interest_coverage,debt_ratio,inventory_turnover,ar_turnover,ap_turnover,asset_turnover,working_capital_turnover,pe_ratio,pb_ratio
symbol,,,,,,,,,,,,,,,,,,,,
AAPL,0.87,0.83,0.17,46.21,31.51,23.97,25.68,164.59,65.34,2.09,15.60,NaN,0.84,NaN,NaN,NaN,1.07,NaN,32.36,49.66
MSFT,1.35,1.35,0.21,68.82,45.62,36.15,16.45,29.65,26.37,0.33,55.49,52.84,0.45,NaN,NaN,NaN,0.46,NaN,38.77,11.36
NVDA,4.44,3.88,0.48,74.99,62.42,55.85,65.30,91.87,90.08,0.13,71.08,341.19,0.29,NaN,NaN,NaN,1.17,NaN,58.50,52.58
AMZN,1.06,0.87,0.44,48.85,10.75,9.29,9.48,20.72,15.94,0.46,45.76,29.52,0.54,NaN,NaN,NaN,1.02,NaN,33.91,7.13
GOOGL,1.84,NaN,0.26,58.20,32.11,28.60,22.24,30.80,33.25,0.08,72.20,448.07,0.28,NaN,NaN,NaN,0.78,NaN,43.50,13.62


_Note: Some efficiency ratios left N/A due to lack of average quarterly values across years._

# DCF Estimation (Fama–French)

This section estimates cost of equity using a simple Fama–French 3-Factor setup (placeholders for factor returns and betas if not available),
then computes a basic WACC and a single-stage DCF using recent free cash flow with a modest growth assumption.
Results are preliminary and for demonstration; refine factors/betas as needed.



In [127]:
# Simple Fama–French-based WACC and DCF (demonstration)
# Minimal assumptions to keep code simple; replace with real factors/betas when available.

# Assumptions
rf = 0.04  # 4% risk-free (placeholder)
market_premium = 0.05  # 5% market premium (placeholder)
SMB_premium = 0.01  # 1% SMB (placeholder)
HML_premium = 0.00  # 0% HML (placeholder)

beta_mkt = 1.1  # fallback beta if missing
beta_smb = 0.2
beta_hml = 0.0

dcf_rows = []
for sym in symbols:
    dat = company_data.get(sym, {})
    ov = dat.get('overview', {}) if isinstance(dat.get('overview', {}), dict) else {}
    bs_ann = (dat.get('bs', {}) or {}).get('annualReports', [])
    cf_ann = (dat.get('cf', {}) or {}).get('annualReports', [])

    bs0 = bs_ann[0] if len(bs_ann) > 0 else {}
    cf0 = cf_ann[0] if len(cf_ann) > 0 else {}

    # Cost of equity via simple FF3
    be_mkt = float(ov.get('Beta') or beta_mkt) if ov.get('Beta') not in (None, 'None', '') else beta_mkt
    be_smb = beta_smb
    be_hml = beta_hml
    cost_of_equity = rf + be_mkt * market_premium + be_smb * SMB_premium + be_hml * HML_premium

    # Cost of debt: interest expense / total debt (fallback to 4.5%)
    total_debt = np.nan
    if bs0.get('shortLongTermDebtTotal') not in (None, 'None', ''):
        total_debt = float(bs0['shortLongTermDebtTotal'])
    else:
        try:
            total_debt = float(bs0.get('longTermDebt') or 0) + float(bs0.get('shortTermDebt') or 0)
        except Exception:
            total_debt = np.nan

    # Use latest income statement quarterly interest if available; else 0
    interest_expense = np.nan
    is_q = (dat.get('income', {}) or {}).get('quarterlyReports', [])
    if is_q:
        ie0 = (is_q[0] or {}).get('interestExpense')
        if ie0 not in (None, 'None', ''):
            try:
                interest_expense = float(ie0)
            except Exception:
                interest_expense = np.nan

    if pd.isna(total_debt) or total_debt == 0 or pd.isna(interest_expense):
        pre_tax_cost_of_debt = 0.045
    else:
        pre_tax_cost_of_debt = max(interest_expense / total_debt, 0.02)

    tax_rate = 0.15  # placeholder effective tax rate
    after_tax_cod = pre_tax_cost_of_debt * (1 - tax_rate)

    # Capital structure weights from balance sheet
    equity_book = float(bs0.get('totalShareholderEquity') or np.nan)
    assets = float(bs0.get('totalAssets') or np.nan)
    debt_book = float(total_debt or 0.0)
    if pd.isna(equity_book) or pd.isna(assets):
        we = 0.7; wd = 0.3
    else:
        # Constrain to [0,1]
        wd = min(max(debt_book / max(debt_book + equity_book, 1.0), 0.0), 1.0)
        we = 1.0 - wd

    wacc = we * cost_of_equity + wd * after_tax_cod

    # Free cash flow proxy (use operating cash flow - capex if available; else fallback to CFO)
    ocf = np.nan
    capex = np.nan
    if cf0:
        try:
            ocf = float(cf0.get('operatingCashflow') or np.nan)
        except Exception:
            ocf = np.nan
        try:
            capex = float(cf0.get('capitalExpenditures') or np.nan)
        except Exception:
            capex = np.nan
    fcf = ocf - capex if not (pd.isna(ocf) or pd.isna(capex)) else ocf

    # Growth assumptions
    g_high = 0.04  # 4% for 5 years
    g_term = 0.025  # 2.5% terminal
    horizon = 5

    if pd.isna(fcf) or fcf <= 0 or wacc <= g_term:
        intrinsic_per_share = np.nan
        note = 'DCF skipped (missing/negative FCF or WACC <= g_term)'
    else:
        # Forecast and discount
        cfs = []
        cf1 = fcf * (1 + g_high)
        for t in range(1, horizon + 1):
            cfs.append(cf1 * ((1 + g_high) ** (t - 1)))
        discounts = [(1 + wacc) ** t for t in range(1, horizon + 1)]
        pv_stage1 = sum(cf / d for cf, d in zip(cfs, discounts))
        terminal_cf = cfs[-1] * (1 + g_term)
        terminal_value = terminal_cf / (wacc - g_term)
        pv_terminal = terminal_value / ((1 + wacc) ** horizon)
        enterprise_value = pv_stage1 + pv_terminal

        # Convert to equity value
        net_debt = debt_book - float(bs0.get('cashAndCashEquivalentsAtCarryingValue') or 0.0)
        equity_value = enterprise_value - net_debt
        shares_out = float(ov.get('SharesOutstanding') or np.nan)
        intrinsic_per_share = equity_value / shares_out if shares_out and shares_out > 0 else np.nan
        note = 'OK'

    # Market price
    mkt_price = np.nan
    if ov.get('MarketCapitalization') and ov.get('SharesOutstanding'):
        try:
            mkt_price = float(ov['MarketCapitalization']) / float(ov['SharesOutstanding'])
        except Exception:
            mkt_price = np.nan

    dcf_rows.append({
        'symbol': sym,
        'cost_of_equity': cost_of_equity,
        'after_tax_cost_of_debt': after_tax_cod,
        'wacc': wacc,
        'fcf_used': fcf,
        'intrinsic_value_per_share': intrinsic_per_share,
        'market_price_per_share': mkt_price,
        'note': note
    })

dcf_table = pd.DataFrame(dcf_rows).set_index('symbol')
with pd.option_context('display.float_format', lambda v: f"{v:,.2%}" if v < 1 and v > 0 else (f"{v:,.2f}")):
    display(Markdown('### DCF Outputs'))
    display(dcf_table)



### DCF Outputs

,cost_of_equity,after_tax_cost_of_debt,wacc,fcf_used,intrinsic_value_per_share,market_price_per_share,note
symbol,,,,,,,
AAPL,10.03%,3.82%,5.83%,"108,807,000,000.00",235.55,220.03,OK
MSFT,9.47%,1.70%,7.56%,"71,611,000,000.00",197.46,524.94,OK
NVDA,14.93%,1.70%,13.41%,"60,853,000,000.00",24.83,180.77,OK
AMZN,10.77%,1.70%,7.92%,"32,878,000,000.00",57.36,223.13,OK
GOOGL,9.27%,1.70%,8.72%,"72,764,000,000.00",219.61,408.46,OK


# Summary Table and Insights

This section combines the ratio and DCF outputs and prints concise notes per company.



In [128]:
# Merge ratios and DCF results
summary = ratios_table.join(dcf_table, how='left')

with pd.option_context('display.float_format', lambda v: f"{v:,.2f}"):
    display(Markdown('### Combined Summary'))
    display(summary)

# Quick textual notes per company
notes = []
for sym, row in summary.iterrows():
    notes.append(f"- {sym}: WACC={row.get('wacc', np.nan):.2%} | P/E={row.get('pe_ratio', np.nan):.2f} | Intrinsic≈{row.get('intrinsic_value_per_share', np.nan):,.2f} vs Price≈{row.get('market_price_per_share', np.nan):,.2f}")

print("Insights:")
for n in notes:
    print(n)



### Combined Summary

,current_ratio,quick_ratio,cash_ratio,gross_profit_margin_pct,operating_profit_margin_pct,net_profit_margin_pct,roa_pct,roe_pct,roce_pct,debt_to_equity,...,working_capital_turnover,pe_ratio,pb_ratio,cost_of_equity,after_tax_cost_of_debt,wacc,fcf_used,intrinsic_value_per_share,market_price_per_share,note
symbol,,,,,,,,,,,,,,,,,,,,,
AAPL,0.87,0.83,0.17,46.21,31.51,23.97,25.68,164.59,65.34,2.09,...,NaN,32.36,49.66,0.10,0.04,0.06,"108,807,000,000.00",235.55,220.03,OK
MSFT,1.35,1.35,0.21,68.82,45.62,36.15,16.45,29.65,26.37,0.33,...,NaN,38.77,11.36,0.09,0.02,0.08,"71,611,000,000.00",197.46,524.94,OK
NVDA,4.44,3.88,0.48,74.99,62.42,55.85,65.30,91.87,90.08,0.13,...,NaN,58.50,52.58,0.15,0.02,0.13,"60,853,000,000.00",24.83,180.77,OK
AMZN,1.06,0.87,0.44,48.85,10.75,9.29,9.48,20.72,15.94,0.46,...,NaN,33.91,7.13,0.11,0.02,0.08,"32,878,000,000.00",57.36,223.13,OK
GOOGL,1.84,NaN,0.26,58.20,32.11,28.60,22.24,30.80,33.25,0.08,...,NaN,43.50,13.62,0.09,0.02,0.09,"72,764,000,000.00",219.61,408.46,OK


Insights:
- AAPL: WACC=5.83% | P/E=32.36 | Intrinsic≈235.55 vs Price≈220.03
- MSFT: WACC=7.56% | P/E=38.77 | Intrinsic≈197.46 vs Price≈524.94
- NVDA: WACC=13.41% | P/E=58.50 | Intrinsic≈24.83 vs Price≈180.77
- AMZN: WACC=7.92% | P/E=33.91 | Intrinsic≈57.36 vs Price≈223.13
- GOOGL: WACC=8.72% | P/E=43.50 | Intrinsic≈219.61 vs Price≈408.46


## Quarterly Timeseries Panel (All Symbols)

Build a full-history quarterly panel across all symbols, then compute ratios per category as time series.


In [129]:
# Build quarterly panel base for all symbols
import pandas as pd, numpy as np

panel_rows = []
for sym in symbols:
    data = company_data.get(sym, {})
    is_q = pd.DataFrame((data.get('income', {}) or {}).get('quarterlyReports', [])).copy()
    bs_q = pd.DataFrame((data.get('bs', {}) or {}).get('quarterlyReports', [])).copy()
    cf_q = pd.DataFrame((data.get('cf', {}) or {}).get('quarterlyReports', [])).copy()

    if is_q.empty or bs_q.empty:
        continue

    # Keep columns
    is_keep = ['fiscalDateEnding','totalRevenue','grossProfit','operatingIncome','ebit','netIncome','costOfRevenue','interestExpense','interestAndDebtExpense']
    bs_keep = ['fiscalDateEnding','totalCurrentAssets','totalCurrentLiabilities','inventory','currentNetReceivables','currentAccountsPayable','totalAssets','totalLiabilities','totalShareholderEquity','cashAndCashEquivalentsAtCarryingValue','cashAndShortTermInvestments','shortLongTermDebtTotal','longTermDebt','shortTermDebt','commonStockSharesOutstanding']
    cf_keep = ['fiscalDateEnding','dividendPayout','dividendPayoutCommonStock']

    for c in is_keep:
        if c not in is_q.columns: is_q[c] = pd.NA
    for c in bs_keep:
        if c not in bs_q.columns: bs_q[c] = pd.NA
    for c in cf_keep:
        if c not in cf_q.columns: cf_q[c] = pd.NA

    is_q = is_q[is_keep].replace({'None': np.nan, '': np.nan})
    bs_q = bs_q[bs_keep].replace({'None': np.nan, '': np.nan})
    cf_q = cf_q[cf_keep].replace({'None': np.nan, '': np.nan})

    # Parse dates
    for df_ in (is_q, bs_q, cf_q):
        df_['fiscalDateEnding'] = pd.to_datetime(df_['fiscalDateEnding'], errors='coerce')
        df_.dropna(subset=['fiscalDateEnding'], inplace=True)
        df_.sort_values('fiscalDateEnding', inplace=True)
        df_.drop_duplicates(subset=['fiscalDateEnding'], keep='last', inplace=True)

    # Coerce numeric
    is_q[[c for c in is_keep if c!='fiscalDateEnding']] = is_q[[c for c in is_keep if c!='fiscalDateEnding']].apply(pd.to_numeric, errors='coerce')
    bs_q[[c for c in bs_keep if c!='fiscalDateEnding']] = bs_q[[c for c in bs_keep if c!='fiscalDateEnding']].apply(pd.to_numeric, errors='coerce')
    cf_q[[c for c in cf_keep if c!='fiscalDateEnding']] = cf_q[[c for c in cf_keep if c!='fiscalDateEnding']].apply(pd.to_numeric, errors='coerce')

    # Merge
    base = bs_q.merge(is_q, on='fiscalDateEnding', how='inner')
    cf_divs = cf_q.copy()
    cf_divs['dividend_total'] = cf_divs[['dividendPayout','dividendPayoutCommonStock']].sum(axis=1, min_count=1)
    base = base.merge(cf_divs[['fiscalDateEnding','dividend_total']], on='fiscalDateEnding', how='left')

    # Derived
    base['cash_equivalents'] = base['cashAndCashEquivalentsAtCarryingValue'].combine_first(base['cashAndShortTermInvestments'])
    base['total_debt'] = base['shortLongTermDebtTotal'].combine_first(base[['longTermDebt','shortTermDebt']].sum(axis=1, min_count=1))
    base['interest'] = base['interestExpense'].combine_first(base['interestAndDebtExpense'])
    base['ebit_eff'] = base['ebit'].combine_first(base['operatingIncome'])
    base['working_capital'] = base['totalCurrentAssets'] - base['totalCurrentLiabilities']
    base['capital_employed'] = base['totalAssets'] - base['totalCurrentLiabilities']

    base = base.sort_values('fiscalDateEnding').reset_index(drop=True)
    base['avg_inventory'] = (base['inventory'] + base['inventory'].shift(1)) / 2.0
    base['avg_ar'] = (base['currentNetReceivables'] + base['currentNetReceivables'].shift(1)) / 2.0
    base['avg_ap'] = (base['currentAccountsPayable'] + base['currentAccountsPayable'].shift(1)) / 2.0
    base['avg_assets'] = (base['totalAssets'] + base['totalAssets'].shift(1)) / 2.0
    base['avg_working_capital'] = (base['working_capital'] + base['working_capital'].shift(1)) / 2.0

    base['symbol'] = sym
    panel_rows.append(base)

if panel_rows:
    ratios_quarterly_panel = pd.concat(panel_rows, ignore_index=True)
    ratios_quarterly_panel.set_index(['symbol', 'fiscalDateEnding'], inplace=True)
    display(Markdown('Built quarterly panel for symbols.'))
else:
    ratios_quarterly_panel = pd.DataFrame()
    display(Markdown('No quarterly panel built (missing data).'))


/var/folders/tc/1rlgym492m70rw7ddhrpc1_80000gn/T/ipykernel_81341/1130625934.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_q = is_q[is_keep].replace({'None': np.nan, '': np.nan})
/var/folders/tc/1rlgym492m70rw7ddhrpc1_80000gn/T/ipykernel_81341/1130625934.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  is_q = is_q[is_keep].replace({'None': np.nan, '': np.nan})
/var/folders/tc/1rlgym492m70rw7ddhrpc1_80000gn/T/ipykernel_81341/1130625934.py:51: FutureWarning: The behavior of array concatenation with empty entries is depr

Built quarterly panel for symbols.

In [130]:
# Timeseries: Quarterly Liquidity Ratios (all symbols)
import pandas as pd, numpy as np

if 'ratios_quarterly_panel' not in globals() or ratios_quarterly_panel.empty:
    raise ValueError('Quarterly panel not built. Run the panel build cell first.')

panel = ratios_quarterly_panel.copy()
liq_ts = pd.DataFrame(index=panel.index)
liq_ts['current_ratio'] = panel['totalCurrentAssets'] / panel['totalCurrentLiabilities']
liq_ts['quick_ratio'] = (panel['totalCurrentAssets'] - panel['inventory']) / panel['totalCurrentLiabilities']
liq_ts['cash_ratio'] = panel['cash_equivalents'] / panel['totalCurrentLiabilities']
liq_ts = liq_ts.replace([np.inf, -np.inf], np.nan)

print('## Quarterly Liquidity Ratios (timeseries)')
print(liq_ts.sort_index())


## Quarterly Liquidity Ratios (timeseries)
                         current_ratio  quick_ratio  cash_ratio
symbol fiscalDateEnding                                        
AAPL   2005-06-30             2.895293     2.833493    0.990714
       2005-09-30             2.861366     2.814007    1.002009
       2005-12-31             2.310474     2.262253    0.820158
       2006-03-31             2.407989     2.362208    1.424147
       2006-06-30             2.286681     2.244276    1.595262
...                                ...          ...         ...
NVDA   2024-04-30             3.529462     3.144255    0.498391
       2024-07-31             4.268953     3.791109    0.613000
       2024-10-31             4.104618     3.640148    0.552643
       2025-01-31             4.439851     3.881310    0.475924
       2025-04-30             3.388403     2.961420    0.573958

[405 rows x 3 columns]


In [131]:
# Timeseries: Quarterly Profitability Ratios (all symbols)
import pandas as pd, numpy as np

if 'ratios_quarterly_panel' not in globals() or ratios_quarterly_panel.empty:
    raise ValueError('Quarterly panel not built. Run the panel build cell first.')

panel = ratios_quarterly_panel.copy()
prof_ts = pd.DataFrame(index=panel.index)
prof_ts['gross_profit_margin_pct'] = (panel['grossProfit'] / panel['totalRevenue']) * 100.0
prof_ts['operating_profit_margin_pct'] = (panel['operatingIncome'] / panel['totalRevenue']) * 100.0
prof_ts['net_profit_margin_pct'] = (panel['netIncome'] / panel['totalRevenue']) * 100.0
prof_ts['roa_pct'] = (panel['netIncome'] / panel['totalAssets']) * 100.0
prof_ts['roe_pct'] = (panel['netIncome'] / panel['totalShareholderEquity']) * 100.0
prof_ts['roce_pct'] = (panel['ebit_eff'] / panel['capital_employed']) * 100.0
prof_ts = prof_ts.replace([np.inf, -np.inf], np.nan)

print('## Quarterly Profitability Ratios (timeseries)')
print(prof_ts.sort_index())


## Quarterly Profitability Ratios (timeseries)
                         gross_profit_margin_pct  operating_profit_margin_pct  \
symbol fiscalDateEnding                                                         
AAPL   2005-06-30                      29.659091                    12.102273   
       2005-09-30                      28.140294                    11.364872   
       2005-12-31                      27.204731                    13.045747   
       2006-03-31                      29.754531                    12.135811   
       2006-06-30                      30.320366                    12.951945   
...                                          ...                          ...   
NVDA   2024-04-30                      78.352020                    64.924743   
       2024-07-31                      75.146471                    62.057257   
       2024-10-31                      74.556753                    62.336811   
       2025-01-31                      73.028908              

In [197]:

# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)  # Auto-detect terminal width
# pd.set_option('display.max_colwidth', None)  # Show full column content
# prof_ts

In [198]:
# Timeseries: Quarterly Solvency Ratios (all symbols)
import pandas as pd, numpy as np

if 'ratios_quarterly_panel' not in globals() or ratios_quarterly_panel.empty:
    raise ValueError('Quarterly panel not built. Run the panel build cell first.')

panel = ratios_quarterly_panel.copy()
solv_ts = pd.DataFrame(index=panel.index)
solv_ts['debt_to_equity'] = panel['total_debt'] / panel['totalShareholderEquity']
solv_ts['equity_ratio_pct'] = (panel['totalShareholderEquity'] / panel['totalAssets']) * 100.0
solv_ts['interest_coverage'] = panel['ebit_eff'] / panel['interest']
solv_ts['debt_ratio'] = panel['totalLiabilities'] / panel['totalAssets']
solv_ts = solv_ts.replace([np.inf, -np.inf], np.nan)

print('## Quarterly Solvency Ratios (timeseries)')
print(solv_ts.sort_index())


## Quarterly Solvency Ratios (timeseries)
                         debt_to_equity  equity_ratio_pct  interest_coverage  \
symbol fiscalDateEnding                                                        
AAPL   2005-06-30              0.077848         65.036232           9.260870   
       2005-09-30              0.106885         64.635097           6.966667   
       2005-12-31              0.073389         59.093153           9.259259   
       2006-03-31              0.114950         62.411042           6.960526   
       2006-06-30                   NaN         61.730846                NaN   
       2006-09-30              0.093550         58.029643           2.623016   
       2006-12-31              0.135465         57.694877          10.492063   
       2007-03-31              0.125031         65.528299           6.662162   
       2007-06-30              0.119591         61.920820           6.716129   
       2007-09-30              0.124209         57.332229           6.235294  

In [199]:
# Timeseries: Quarterly Efficiency Ratios (all symbols)
import pandas as pd, numpy as np

if 'ratios_quarterly_panel' not in globals() or ratios_quarterly_panel.empty:
    raise ValueError('Quarterly panel not built. Run the panel build cell first.')

panel = ratios_quarterly_panel.copy()
eff_ts = pd.DataFrame(index=panel.index)
eff_ts['inventory_turnover'] = panel['costOfRevenue'] / panel['avg_inventory']
eff_ts['ar_turnover'] = panel['totalRevenue'] / panel['avg_ar']
eff_ts['ap_turnover'] = panel['costOfRevenue'] / panel['avg_ap']
eff_ts['asset_turnover'] = panel['totalRevenue'] / panel['avg_assets']
eff_ts['working_capital_turnover'] = panel['totalRevenue'] / panel['avg_working_capital']
eff_ts = eff_ts.replace([np.inf, -np.inf], np.nan)

print('## Quarterly Efficiency Ratios (timeseries)')
print(eff_ts.sort_index())


## Quarterly Efficiency Ratios (timeseries)
                         inventory_turnover  ar_turnover  ap_turnover  \
symbol fiscalDateEnding                                                 
AAPL   2005-06-30                       NaN          NaN          NaN   
       2005-09-30              1.476536e+01     3.048487     1.597461   
       2005-12-31              2.046455e+01     3.086711     1.790374   
       2006-03-31              1.366964e+01     1.982715     1.223821   
       2006-06-30              1.460432e+01     2.844126     1.317897   
       2006-09-30              1.418219e+01     2.745837     1.160427   
       2006-12-31              1.708551e+01     2.732860     1.345704   
       2007-03-31              1.336595e+01     2.202510     1.086196   
       2007-06-30              1.488017e+01     2.368651     1.126505   
       2007-09-30              1.382580e+01     1.794228     0.956431   
       2007-12-31              1.559255e+01     2.273814     1.214396   
       

In [200]:
# Timeseries: Quarterly Market Ratios (limited; all symbols)
import pandas as pd, numpy as np

if 'ratios_quarterly_panel' not in globals() or ratios_quarterly_panel.empty:
    raise ValueError('Quarterly panel not built. Run the panel build cell first.')

panel = ratios_quarterly_panel.copy()
# Coerce needed columns to numeric
for c in ['netIncome','commonStockSharesOutstanding','dividend_total']:
    if c in panel.columns:
        panel[c] = pd.to_numeric(panel[c], errors='coerce')

mkt_ts = pd.DataFrame(index=panel.index)
mkt_ts['eps'] = panel['netIncome'] / panel['commonStockSharesOutstanding']
mkt_ts['dividend_payout_ratio_pct'] = (panel['dividend_total'] / panel['netIncome']) * 100.0
mkt_ts = mkt_ts.replace([np.inf, -np.inf], np.nan)

print('## Quarterly Market Ratios (timeseries, limited)')
print(mkt_ts.sort_index())


## Quarterly Market Ratios (timeseries, limited)
                              eps  dividend_payout_ratio_pct
symbol fiscalDateEnding                                     
AAPL   2005-06-30        0.013235                        NaN
       2005-09-30        0.017567                        NaN
       2005-12-31        0.023082                        NaN
       2006-03-31        0.016667                        NaN
       2006-06-30        0.019235                        NaN
       2006-09-30        0.022088                        NaN
       2006-12-31        0.040595                        NaN
       2007-03-31        0.031016                        NaN
       2007-06-30        0.032800                        NaN
       2007-09-30        0.035871                        NaN
       2007-12-31        0.062734                        NaN
       2008-03-31        0.041499                        NaN
       2008-06-30        0.042391                        NaN
       2008-09-30        0.045005   

Ranking


Backtest

## Strategy Backtest: Factor Composite (EPS Momentum, Value, ROE, Leverage, Price Filter)
This section builds a quarterly-rebalanced backtest using your existing `symbols`, `API_KEY`, and `ratios_quarterly_panel`.

- Universe: current `symbols` (demo scale; full NASDAQ requires batching/caching)
- Signals and thresholds:
  - EPS TTM Growth YoY > 25% → +1; < 0% → -1; else 0
  - Value: P/E < 15 OR P/B < 1.5 → +1; P/E > 30 AND P/B > 3 → -1; else 0
  - ROE > 20% → +1; ROE < 5% → -1; else 0
  - Debt-to-Equity < 0.5 → +1; > 1.5 → -1; else 0
  - Price > 200D SMA → +1; Price < 200D SMA → -1; else 0
  - Insider buying: 0 (placeholder)
- Selection: score ≥ 4
- Rebalance: quarterly on available fiscal dates


In [ ]:
# Backtest: download prices and compute 200D SMA
import pandas as pd, numpy as np, time, requests

if 'symbols' not in globals() or not symbols:
    symbols = ['AAPL','MSFT','NVDA','AMZN','GOOGL']

if 'API_KEY' not in globals():
    raise ValueError('API_KEY not defined')

prices = {}
for sym in symbols:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={sym}&outputsize=full&apikey={API_KEY}'
    r = requests.get(url).json()
    ts = r.get('Time Series (Daily)', {})
    if not ts:
        print(f'No price data for {sym}')
        continue
    df = pd.DataFrame(ts).T
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df.rename(columns={'5. adjusted close': 'adj_close'})
    df['adj_close'] = pd.to_numeric(df['adj_close'], errors='coerce')
    df['sma_200'] = df['adj_close'].rolling(200).mean()
    prices[sym] = df[['adj_close','sma_200']]
    time.sleep(12)  # respect AV rate limits

print('Downloaded prices and computed 200D SMA.')


In [ ]:
# Backtest: construct quarterly factor features from panel and prices
import pandas as pd, numpy as np

if 'ratios_quarterly_panel' not in globals() or ratios_quarterly_panel.empty:
    raise ValueError('ratios_quarterly_panel not available; build it first.')

panel = ratios_quarterly_panel.copy()
panel = panel.sort_index()

# Helper: compute TTM EPS and YoY growth
def compute_eps_ttm(df):
    eps_q = df['netIncome'] / df['commonStockSharesOutstanding']
    eps_ttm = eps_q.rolling(4).sum()
    eps_ttm_yoy = eps_ttm.pct_change(4) * 100.0
    return eps_ttm, eps_ttm_yoy

# Build per-symbol features on quarter-end dates
features = []
for sym, g in panel.groupby(level=0):
    g = g.copy()
    # ensure needed fields are numeric
    for c in [
        'netIncome','commonStockSharesOutstanding','totalRevenue','totalAssets',
        'totalShareholderEquity','total_debt','totalLiabilities','capital_employed'
    ]:
        if c in g.columns:
            g[c] = pd.to_numeric(g[c], errors='coerce')
    g['eps_ttm'], g['eps_ttm_yoy_pct'] = compute_eps_ttm(g)

    # Valuation: P/E and P/B from latest close aligned to quarter date
    px = prices.get(sym)
    if px is None or px.empty:
        g['pe'] = np.nan
        g['pb'] = np.nan
        g['price_above_200d'] = np.nan
    else:
        # align price on or after quarter end (next trading day)
        q_dates = g.index.get_level_values(1)
        # reindex price to include quarter dates via asof
        joined = px.reindex(px.index.union(q_dates)).sort_index()
        adj_on_q = joined['adj_close'].ffill().loc[q_dates]
        sma_on_q = joined['sma_200'].ffill().loc[q_dates]
        g.loc[(sym, slice(None)), 'price'] = adj_on_q.values
        g.loc[(sym, slice(None)), 'sma_200_on_q'] = sma_on_q.values
        g['pe'] = g['price'] / g['eps_ttm']
        # book value per share approximation: equity / shares
        bvps = g['totalShareholderEquity'] / g['commonStockSharesOutstanding']
        g['pb'] = g['price'] / bvps
        g['price_above_200d'] = (g['price'] > g['sma_200_on_q']).astype(float)

    # Profitability / leverage
    g['roe_pct'] = (g['netIncome'] / g['totalShareholderEquity']) * 100.0
    g['de_ratio'] = g['total_debt'] / g['totalShareholderEquity']

    keep = g[[
        'eps_ttm_yoy_pct','pe','pb','roe_pct','de_ratio','price_above_200d'
    ]].copy()
    keep['symbol'] = sym
    keep['date'] = keep.index.get_level_values(1)
    features.append(keep.reset_index(drop=True))

features_df = pd.concat(features, ignore_index=True) if features else pd.DataFrame()
features_df = features_df.dropna(how='all')
print('Built quarterly factor features for backtest.')


In [ ]:
# Backtest: scoring, selection, and daily portfolio returns
import pandas as pd, numpy as np

if 'features_df' not in globals() or features_df.empty:
    raise ValueError('features_df not available; build features first.')

f = features_df.copy()
# Fill NaNs for robust scoring
for c in ['eps_ttm_yoy_pct','pe','pb','roe_pct','de_ratio','price_above_200d']:
    if c not in f.columns:
        f[c] = np.nan

# Scores per rules
f['score_eps'] = np.where(f['eps_ttm_yoy_pct'] > 25, 1, np.where(f['eps_ttm_yoy_pct'] < 0, -1, 0))
val_pos = (f['pe'] < 15) | (f['pb'] < 1.5)
val_neg = (f['pe'] > 30) & (f['pb'] > 3)
f['score_val'] = np.select([val_pos, val_neg],[1, -1], default=0)
f['score_roe'] = np.where(f['roe_pct'] > 20, 1, np.where(f['roe_pct'] < 5, -1, 0))
f['score_de'] = np.where(f['de_ratio'] < 0.5, 1, np.where(f['de_ratio'] > 1.5, -1, 0))
px_pos = f['price_above_200d'] >= 1
px_neg = f['price_above_200d'] == 0
f['score_px'] = np.select([px_pos, px_neg],[1, -1], default=0)

f['total_score'] = f[['score_eps','score_val','score_roe','score_de','score_px']].sum(axis=1)

# Quarterly rebalance dates
rebalance_dates = sorted(f['date'].dropna().unique())

# Build daily portfolio returns between rebalances
all_daily = []
period_summaries = []
for i, d in enumerate(rebalance_dates[:-1]):
    next_d = rebalance_dates[i+1]
    universe_slice = f[f['date'] == d].dropna(subset=['symbol'])
    picks = universe_slice[universe_slice['total_score'] >= 4]['symbol'].unique().tolist()
    if not picks:
        # carry zero return period
        # still build an index using a benchmark symbol if available
        any_px = prices.get(symbols[0]) if 'prices' in globals() and symbols else None
        if any_px is not None:
            # use date bounds to slice, but returns set to 0
            start_ix = any_px.index.searchsorted(pd.to_datetime(d), side='left')
            end_ix = any_px.index.searchsorted(pd.to_datetime(next_d), side='left')
            idx = any_px.index[start_ix:end_ix]
            if len(idx) > 0:
                z = pd.Series(0.0, index=idx, name='ret')
                all_daily.append(z)
        period_summaries.append({'date': d, 'num_picks': 0, 'period_return': 0.0})
        continue

    # Construct daily return series for each pick
    sym_daily = []
    for sym in picks:
        px = prices.get(sym)
        if px is None or px.empty:
            continue
        start_ix = px.index.searchsorted(pd.to_datetime(d), side='left')
        end_ix = px.index.searchsorted(pd.to_datetime(next_d), side='left')
        # slice half-open [start, end)
        window = px.iloc[start_ix:end_ix]
        if window.empty:
            continue
        r = window['adj_close'].pct_change().fillna(0.0)
        r.name = sym
        sym_daily.append(r)
    if not sym_daily:
        period_summaries.append({'date': d, 'num_picks': 0, 'period_return': 0.0})
        continue

    R = pd.concat(sym_daily, axis=1).dropna(how='all')
    # equal-weight
    port_r = R.mean(axis=1).rename('ret')
    all_daily.append(port_r)

    period_ret = (1.0 + port_r).prod() - 1.0
    period_summaries.append({'date': d, 'num_picks': len(picks), 'period_return': float(period_ret)})

portfolio_daily = pd.concat(all_daily).sort_index() if all_daily else pd.Series(dtype=float)
period_summary_df = pd.DataFrame(period_summaries)

print('Built scores, selections, and daily portfolio returns.')
print(period_summary_df.head())


In [ ]:
# Backtest: performance statistics and prints
import pandas as pd, numpy as np

if 'portfolio_daily' not in globals() or portfolio_daily.empty:
    raise ValueError('No daily portfolio returns computed.')

ret = portfolio_daily.copy().astype(float)
ret = ret[ret.index.notna()]

# CAGR
years = (ret.index[-1] - ret.index[0]).days / 365.25
cum_return = (1.0 + ret).prod() - 1.0
cagr = (1.0 + cum_return) ** (1.0 / max(years, 1e-9)) - 1.0

# Max drawdown
cum = (1.0 + ret).cumprod()
peak = cum.cummax()
drawdown = cum / peak - 1.0
max_dd = drawdown.min()

# Sharpe (daily to annualized)
sharpe = (ret.mean() / (ret.std() + 1e-12)) * np.sqrt(252)

# Sortino (downside std)
downside = ret.copy()
downside[downside > 0] = 0
sortino = (ret.mean() / (downside.std() + 1e-12)) * np.sqrt(252)

stats = {
    'CAGR': float(cagr),
    'Cumulative Return': float(cum_return),
    'Max Drawdown': float(max_dd),
    'Sharpe': float(sharpe),
    'Sortino': float(sortino),
    'Start': str(ret.index[0].date()),
    'End': str(ret.index[-1].date()),
    'Num trading days': int(ret.shape[0])
}

print('## Backtest Performance')
for k, v in stats.items():
    print(f'{k}: {v}')

if 'period_summary_df' in globals():
    print('\nPeriod summaries (first 10):')
    print(period_summary_df.head(10))
